**PySDS Week 3 Lecture 2. V.2**
Last author: B. Hogan

# Week 3. Day 2. Exploratory Data Analysis 

**Update on Twitter**
Today we would first like to introduce exploratory data analysis as a topic. This is somewhat of a departure from the expected lecture on Twitter. This is on account of the pending developer API key request. I had hoped that Twitter would have responded by now for this request after several days. Further, the experience has made me particularly concerned for the potential for students to be able to register their own applications in a tidy fashion. 

Today, students should, in class with the instructor walk through the process of applying for a developer key for personal use. We will see if / when these are approved for individual work. In the meantime, we will be using the existing Twitter data that's available for merging tasks.  


# Section 1. Exploratory data analysis 

Data analysis in most work involves building up a case in stages. Many people in their analysis miss the importance and the care involved in the early stages of data analysis. This work is important for a variety of reasons. Even before we are testing relationships or even classifying data, we will want to have a look at various aspects of the data for their own sake. We want to 'get to know' our data. 

In many papers, there is a common theme where work expands from univariate to bivariate to multivariate. This course will not involve specific multivariate relationships, however, in order to accomplish tasks in data wrangling, it is common to have the capacity for rapid querying of data filtered, sorted, and grouped in a variety of ways. As we could see from the merging formative assignment from last week, there was lots to be learned from grouping and sorting data. There are also many ways to accomplish these tasks, both in terms of algorithmic implementation and simple matters of style and syntax. At the same time, there are some fundamentals that don't change such as whether we want to transform data, when should we filter that data, delete outliers, etc... 

Today, we will walk through an exploratory data analysis of the May5-6 Twitter Data set. The version of this database used last week only had the root tweets. This week we will see the replies. 

We will also have a slight diversion to look at how the replies were captured outside the Twitter API. We will return to this again next week when discussing best practices and ethical research on the web. 



## Step 1. What variables are available, and in what form? 

When we take in data from an SQL table or a json file, it might not be in the form that we want. For example, are the dates in a datetime format? Is the text in the appropriate encoding. 

First we want to describe each variable in our dataframe if possible. As we have seen from the Reddit exercise yesterday, there are a variety of possible data points that are available. We also saw that there is an endless amount of potential data to process and combine. That being said, at any given point you will likely be dealing with a finite and estimable slice of a larger amount of data. 

You should also consider theory in your data processing matters. That is to say, the use of a specific slice of data should be driven by particular research motivations. New techniques in machine learning do not overturn this basic matter. Instead, they provide increasing levels of scale as well as the ability to classify documents according to patterns that may be observable and intuitive but not easily and clearly specified. But even in the case of advanced methods it is important to have data well documented and prepared ahead of time. 

## Step back 1. What level of theory is relevant? 

There are some disciplines where theory is both rigorously specified and quite limiting. Experimental psychology, for example, is a discipline that is focused intently on rigor and on theories in a range that allow for very coherent and bounded specification. Political sociology, on the other hand, must contend with a variety of only vaguely specified phenomena that are easy to identify in an intuitive sense but often hard to quantify. In social data science, we tend to focus on the act of operationalisation. That is, we seek to let some form of measurement stand in for some phenomenon. We then report on that measurement. Theory comes into the selection of the space as well as the selection of topics within that space. What it does not tend to do, however, is specify the specific mechanism that is being used. Rather, it triangulates mechanisms through a variety of approaches, from A/B testing to k-fold cross validation sampling. So in data science, you are less likely to hear about how correlation does not equal causation. Instead, you are more likely to be asked what are your benchmarks or what is your 'ground truth' and thereafter how is your performance? Theory is about understanding how a particular set of expectations or measurements can serve as benchmarks, expectations or ways to determine both ground truth and deviations from it. 

## Returning to the variables: With a theory

Instead of selecting all of the variables available, it is useful to look at the specific variables of interest, winnow that set down to a manageable number and then report on some of the important descriptive statistics for these variables. Now you might think that new techniques, if you have heard of them, like "LASSO" and "LDA" could spell the end of such prudence. We can throw in all the variables, tweak a few parameters and out comes some results. Or that once a system is running, we can tweak a few variables in a platform, compare the results between the two and make a decision. In both cases, it pays to be careful. To paraphrase: we miss 100% of the variables we don't save. It is also the case that in many domains we cannot simply scale up features or variables. If we have $n$ Facebook like datapoints adding $m$ more data  points doesn't really change the algorithm, it only increases the available data points, in fact, it might increase them too much. But it does not help us with the variety of other types of data points with different qualities. This we might only reach a local optimization and miss out on a big breakthrough. 

### Display your variables

It is useful to display each and every variable you plan on using in your dataset. How you display them depends on the type of data. For all data, you will be able to display 

```
name: <var name>
type: <var type>
count: <var count> 
count valid: <count - missing> 
```

Iterating through all of your variables and printing this out can become the basis of a codebook. In social science data analysis, we often have concpetual distinctions between numeric data based on whether the data has a natural zero or not. For example, IQ does not have a natural zero. It is based on deviation from a mean value. So an IQ of 100 refers to the mean score on the IQ test, and numbers above and below refer to some deviation from that middle point. For many IQ rankings 10 points refers to 1 standard deviation. This distinction tends to be less relevant in data science as the values are less commonly from self reported data (where we emphasize the mean) and more from trace data, where we tend to count things rather than measure their deviation from an average. The order of measurements tends to be:

- **Nominal / Categorical**. There is no obvious ranking, only classification
 - Think religion or favorite color. There's no obvious ranking only preferences and distributions.
- **Ordinal**. There is a ranking but the distance between the ranking is ambiguous. 
 - Think attitudinal statements such as "I strongly agree with the following statement..."
- **Interval**. There is a ranking and the distance is meaningful but there's no obvious zero
 - IQ scores, likert questions, others where score is derived from a distribution of values. 
- **Ratio**. There is a ranking, distance is meaningful and there is a zero.
 - Messages, signals, traces where we can count the number of these and a case can have zero.
 
When we want to check on data and report on it, different variable types will correspond to different kinds of statistics being more or less useful. In a difference parlance, this refers to **parametric** versus **non-parmetric** statistics. In non-parametric statistics we tend to simply be concerned with the rank order of entries in a distribution, whereas in parametric statistics the distance between the elements in a distribution make sense. We tend to use non-parametric statistics when we have skewed data and do not want to, or are not able to fit a specific distribution to a set of data. 

Below lets look at the variables in the roottweets table, then examine the distributions.  

In [ ]:
import pandas as pd 
import sqlite3

df = pd.read_sql("select * from roottweets",sqlite3.connect("PySDS_ElectionData_2015_may5-6withReplies.db"))
df.head()

To get some basic statistics on the data we can use the describe command. However, we are going to create our own here, so that we can learn a little more, mix categorical and numeric values and have it presented in a different axis. It also will help you learn how to build your own functions. 

In [ ]:
df.describe()

In [ ]:
def createCodebookTable(df):
    '''Turn a series of columns into a basic codebook showing type, max,min and count variables.'''
    df_codebook = pd.DataFrame(index=df.columns, columns=["dtype","type","min","max","count","missing"])
    df_codebook["dtype"] = [str(df[x].dtype) for x in df.columns]
    df_codebook["type"] = [str(type(df[x][0])) for x in df.columns]
    
    df_codebook["min"] = df.min() 
    df_codebook["max"] = df.max() 
    df_codebook["count"] = df.count()
    df_codebook["missing"] = df.isnull().sum()
    return df_codebook

display(createCodebookTable(df))

Okay, so it seems that don't yet have our variables in a format that is useful to us. To do that, we should inspect them and convert them to any proper format. For example, the tweet_ID should be in a long format, the date should be in a date format. We can parse these values in order to make better use of them. Both ```text``` and ```username``` came down in the correct format of 'string'. 

In [ ]:
from datetime import datetime 

df["tweet_id"] = df["tweet_id"].map(lambda x: int(x))
display(createCodebookTable(df))

# Now it's your turn

# df["date"] = xx
# "2015-05-05 00:00:07.000000"

# "%Y-%m-%d %H:%M:%S.000000"

df["date"] = df["date"].map(lambda x: x[:-7])
df["date"] = df["date"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %X.%f'))
display(createCodebookTable(df))



# Adding new variables to our dataframe. 

Once we have cleaned up our dataframe we will want to either add new variables to our dataframe, filter out data we do not want or group that data. We will repeat this process again when we merge the data in with other data. 

We will add several new variable sets to our dataframe here. 
1. A number and list of @mentions. 
2. Length of the tweet. 
3 .The number and list of emoji in our tweets. 
 - Note: To do the third one we will first need to install the external ```emoji``` library. 

In [ ]:
# Variable set 1. The number of @mentions. 
import re
regex = re.compile("@\w*")

df["atmention_count"] = df["text"].map(lambda x: len(set(regex.findall(x))))
df["atmention_list"] = df["text"].map(lambda x: list(set(regex.findall(x))))

df.head()

Notice that here for the first time, we have inserted a list inside the table. As noted, a DataFrame is a collection of objects, and a list itself is an object. This might cause is some trouble down the road. What's a min value for a list? 

In [ ]:
display(createCodebookTable(df))

So min is the empty set and max is alphabetical. Not the most useful, but we still get to see the class of data, the valid count and the datatype. 

In [ ]:
# Variable set 2. The length of the tweet.

df["tweet_len"] = df["text"].map(lambda x: len(x))
display(createCodebookTable(df))

So this shows the max value; it makes it clear that this was before Twitter's expansion into 240 characters. But isn't Twitter supposed to be up to 140 characters? I believe that this does not include characters when they are embedded in links. Let's explore this further. 

In [ ]:
s_max_tweets = df[df["tweet_len"]>160]["text"]

for i in s_max_tweets: 
    print(i,"\n---\n")

It would appear that these tweets tend to have links in them which might help to explain the length. It is also the case that there are a few other artifacts worth discovering. For example the tweets appear to have ```&amp;``` in them. That is an html code for the & symbol. Why don't we clean that out? 

In [ ]:
import html
df['text'] = df['text'].map(lambda x: html.unescape(x))

# And now I'm going to re-run the length code to see if it made a difference
df["tweet_len"] = df["text"].map(lambda x: len(x))
display(createCodebookTable(df))

It appears it did.

In [ ]:
s_max_tweets = df[df["tweet_len"]>160]["text"]

for i in s_max_tweets: 
    print(i,"\n---\n")

Now we see a pattern here as well. It appears that these are all retweets and coded as such. Are they the only retweets? Well, no, they are the retweets where people both retweeted the maximum length twitter handle and then used the maximum length of the text. But this suggests an additional feature - "is retweet". This feature was not introduced before because it was not obvious how this could be handed with our data. Now it appears that we can do this using this structure. Does the original data have 'is retweet' as a boolean? We cannot recover that information from here, so we work with what we have. 

In [ ]:
import re

def isRetweet(text):
    '''A method to determine if the text prefix is RT
    
    This is a regular expression.
    '''
    if text[0:2].lower() != 'rt':
        return (False,None)
    else:
        x = re.compile("RT @\w*\:")
        y = x.findall(text)
        if len(y) > 0:
            return (True,y[0][4:-1])
        else:
            return (False,None)
        
df["is_retweet"] = df["text"].map(lambda x: isRetweet(x)[0])
df["retweet_original"] = df["text"].map(lambda x: isRetweet(x)[1])

df.head()

In [ ]:
display(createCodebookTable(df))

# Variable set 3. Emoji

Emoji can be highly dimensional. We can ask not just the number and kind of emoji, but the variety. Here, we will simply get the count and then all the emojis concatenated in a list for later processing. 

In [ ]:
try: 
    import emoji
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install emoji
    import emoji

In [ ]:
dir(emoji)

In [ ]:
?emoji.emoji_lis

In [ ]:
test_text = " Hi, I am fine. 😁: '😁' "

import emoji

def getEmojiCount(text):
    result = emoji.emoji_lis(text)
    emoji_list = []
    
    for i in result:
        emoji_list.append(i["emoji"])
        
        
    return (len(emoji_list),emoji_list)


print(getEmojiCount(test_text))

In [ ]:
# df["emoji_count"] = df["text"].map(lambda x: getEmojiCount(x)[0])
# df["emoji_list"] = df["text"].map(lambda x: getEmojiCount(x)[1])

df[df["emoji_count"] > 2].head(25)

In [ ]:
display(createCodebookTable(df))

# Finally, let's try out a sentiment analysis tool. 

The tool being used today is VADER, by C. J. Hutto and Eric Gilbert. It is designed for small text corpus. You can see VADER discussed here: https://github.com/cjhutto/vaderSentiment. The read me page points to the academic paper, the data sets and discusses the use of the tool. 

In [ ]:
try: 
    import vaderSentiment
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install vaderSentiment
    import vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sentarr = []
    
for c, row in df.iterrows(): 
    vs = analyzer.polarity_scores(row["text"])
    sentarr.append([row["tweet_id"], vs['neg'],vs['pos']])
    
#     print("{:-<65} {}".format(tweet, str(vs)))       


In [ ]:
sent_df = pd.DataFrame(sentarr, columns = ["tweet_id","neg","pos"])
df = df.merge(sent_df,on="tweet_id")

**In class exercise:** We have seen now that VaderSentiment analyzer returns negative, neutral, positive and compound scores. From the demo snippet that we saw on the website, we see that ```vs``` is what is returned from ```polarity_scores(<text>)```. Now, having seen the above examples, create four new columns, each with the corresponding header and insert the data values in there. 